In [1]:
import time
import tempfile
import numpy as np
import tensorflow as tf
print('Version:', tf.__version__)

Version: 1.15.3


In [2]:
tf.enable_eager_execution()

# Tensors

In [3]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64('hello_world'))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG9fd29ybGQ', shape=(), dtype=string)


In [4]:
# get result with numpy() method...
print(tf.add(1,2).numpy())

3


In [5]:
# operator overloading
print(tf.square(2) + tf.square(3))

tf.Tensor(13, shape=(), dtype=int32)


#### NumPy Compatibility

Major Differences Between NumPy arrays and TensorFlow tensors:

1) Tensors can be backed by accelerator memory (GPU, TPU)

2) Tensors are immutable

TensorFlow operations automatically convert NumPy ndarrays to Tensors.

NumPy operations automatically convert Tensors to NumPy ndarrays.

In [6]:
npy_array = np.ones([3, 3])

tensor = tf.multiply(npy_array, 42)
print(type(tensor))
print(tensor)

array = np.add(tensor, 1)
print(type(array))
print(array, array.shape, array.dtype)

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
<class 'numpy.ndarray'>
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]] (3, 3) float64


In [7]:
# explicit conversion
print(type(tensor.numpy()))
print(type(tf.convert_to_tensor(array)))

<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


# GPU acceleration

In [8]:
# if not specified, TensorFlow will automatically decide whether or not to use GPU or CPU for an operation
x = tf.random.uniform([3, 3])

print('Is there a GPU?')
print(tf.test.is_gpu_available())

print('Is Tensor on GPU?')
print(x.device.endswith('GPU:0'))

Is there a GPU?
True
Is Tensor on GPU?
True


#### Names and Placement
TensorFlow operations can be explicitly placed on specific devices using the tf.device context manager.

In [9]:
# tensor.device provides fully qualified string name of device hosting contents of tensor
print(x.device) # required for distributed execution

/job:localhost/replica:0/task:0/device:GPU:0


In [10]:
# example operation for demonstration
def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
    runtime = time.time() - start
    print('10 loops: {:0.2f}ms'.format(1000 * runtime))

In [11]:
# force execution on CPU
print('Running on CPU...')
with tf.device('CPU:0'):
    # initialize large random matrix
    x = tf.random_uniform([1000, 1000])
    # confirm CPU or throw error
    assert x.device.endswith('CPU:0')
    time_matmul(x)

Running on CPU...
10 loops: 80.82ms


In [12]:
# force execution on GPU
print('Running on GPU...')
with tf.device('GPU:0'):
    # initialize large random matrix
    x = tf.random_uniform([1000, 1000])
    # confirm GPU or throw error
    assert x.device.endswith('GPU:0')
    time_matmul(x)

Running on GPU...
10 loops: 167.87ms


# Datasets
When eager execution is enabled you can iterate over a dataset object without creating a tf.data.Iterator object.

In [13]:
# create dataset from tensor slices
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

In [14]:
# create mock CSV file
_, filename = tempfile.mkstemp()
with open(filename, 'w') as f:
    f.write('''Line 1.1\nLine 1.2\nLine 1.3''')
    
# create dataset from file
ds_file = tf.data.TextLineDataset(filename)

In [15]:
# apply transformations
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)
ds_file = ds_file.batch(1)

In [16]:
# iterate over dataset
for x in ds_tensors: print(x)
for x in ds_file: print(x)

tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([16 25], shape=(2,), dtype=int32)
tf.Tensor([36  9], shape=(2,), dtype=int32)
tf.Tensor([b'Line 1.1'], shape=(1,), dtype=string)
tf.Tensor([b'Line 1.2'], shape=(1,), dtype=string)
tf.Tensor([b'Line 1.3'], shape=(1,), dtype=string)
